# Player evaluation

Here we compare players based on their adjusted points. Also, we calculate some useful metrics using the adjusted points to further evaluate player performance in FPL.

In [1]:
# recent gameweek
gameweek = 20

In [2]:
# import basic libraries
import pandas as pd
import numpy as np

pd.set_option('max_columns',100)

In [3]:
# fetch FPL data
filepath = '../data/data_week' + str(gameweek) + '.csv'
df = pd.read_csv(filepath, index_col=0, encoding='latin-1')

# fetch team data
filepath = '../data/fbref/team_stats_week' + str(gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col=0, encoding='latin-1')

# fetch player stats data
filepath = '../data/fbref/player_stats_week' + str(gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1, encoding='latin-1')

## Value and value points

Here we calculate two interesting metrics: 'value' and 'value points'. 

Value is simply adjusted points per game divided by the cost of the player. Essentially, this measures how many 'points per pound(/euro/whatever)' has the player gained on an average game week. 

The value points is calculated as the geometric mean of a players adjusted points per game and value. Essentially, in FPL we want to gain the maximum possible amount of points. However, we are restrained by a limited budget, so that in practice we cannot just pick all the best players. Then, we would like to have a squad of players that give the most points given our budget. Thus, we would like to pick players that have high value ('bang for buck'). However, often the players with highest value are very cheap players who somewhat overperform relative to their price. Picking a squad full of these players might leave a part of our budget unused. Calculating the geometric mean of adjusted points and value allows us to weigh both aspects equally, i.e. we want players that both gain a lot of points but also have good value. For example, this metric values equally two players, where one has twice the points per game of the other who in turn has twice the value of the other.

In [4]:
# value = expected points / cost
df['value'] = df['adjusted points per game'] / (df['now_cost'] / 10.0)
# geometric mean of 'adjusted points per game' and 'value'
df['valuePoints metric'] = np.sqrt(df['adjusted points per game'] * df['value'])

In [5]:
# assign proper team names for each player
team_names = np.sort(teamStats['Squad'])
df['team_name'] = team_names[df['team']-1]

In [6]:
# save data
filepath = '../data/data_week' + str(gameweek) + str('.csv')
df.to_csv(filepath)

In [7]:
goalkeepers = df['element_type'] == 1
defenders = df['element_type'] == 2
midfielders = df['element_type'] == 3
forwards = df['element_type'] == 4

minGames = df['games played'] >= 3

## Player evaluation

Below we compile lists for each position sorting players based on a given metric.

In [10]:
#metric = 'valuePoints metric'
metric = 'adjusted points per game'
#metric = 'form 5'

numberToShow = 40

print('GOALKEEPERS')
display(df[goalkeepers & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))
print('DEFENDERS')
display(df[defenders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))
print('MIDFIELDERS')
display(df[midfielders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))
print('FORWARDS')
display(df[forwards & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))

GOALKEEPERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
411,PatrÃÂ­cio,Wolves,52,20.000000,76,88.276375,3.8,4.413819,4.532485,1.935588
388,Fabianski,West Ham,49,8.000000,40,33.044848,5.0,4.130606,8.445227,1.866018
235,de Gea,Manchester Utd,54,20.000000,62,81.094366,3.1,4.054718,3.786200,1.744873
471,Henderson,Sheffield Utd,49,18.809524,79,75.066824,4.2,3.990894,5.092929,1.802902
131,Guaita,Crystal Palace,51,18.095238,76,71.335508,4.2,3.942225,5.368922,1.745647
340,Lloris,Tottenham,53,6.969697,23,27.313379,3.3,3.918876,NaN,1.702251
47,Ryan,Brighton,48,20.000000,78,77.144714,3.9,3.857236,3.995082,1.760579
494,Ramsdale,Bournemouth,47,20.000000,76,76.503797,3.8,3.825190,4.341664,1.764429
168,Schmeichel,Leicester City,54,20.000000,78,74.455782,3.9,3.722789,2.643449,1.602033


DEFENDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
182,Alexander-Arnold,Liverpool,74,18.928571,106,109.723582,5.6,5.796718,7.222161,2.130916
297,Lundstram,Sheffield Utd,52,19.019608,97,97.537238,5.1,5.128247,5.323665,2.248885
181,Robertson,Liverpool,70,19.111111,86,95.623387,4.5,5.003549,4.468824,1.891164
401,Doherty,Wolves,61,18.000000,72,79.263260,4.0,4.403514,3.957828,1.782931
185,Matip,Liverpool,52,6.956522,32,30.277559,4.6,4.352399,NaN,1.908653
103,Alonso,Chelsea,60,7.966102,47,34.562261,5.9,4.338667,6.851449,1.771253
183,van Dijk,Liverpool,64,18.888889,85,79.978649,4.5,4.234164,4.325120,1.673700
208,Otamendi,Manchester City,50,13.142857,46,52.181664,3.5,3.970344,3.522561,1.775592
405,Boly,Wolves,48,8.095238,34,32.083373,4.2,3.963240,NaN,1.808963


MIDFIELDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
191,Salah,Liverpool,123,15.942029,110,105.599215,6.9,6.623951,8.412237,1.888707
192,ManÃÂ©,Liverpool,123,18.082192,132,117.803911,7.3,6.514913,6.936672,1.857617
215,De Bruyne,Manchester City,105,19.041096,139,122.404331,7.3,6.428429,6.226855,1.983856
214,Sterling,Manchester City,118,19.122807,109,112.176149,5.7,5.866092,5.245458,1.707685
239,Martial,Manchester Utd,79,14.107143,79,70.597681,5.6,5.004393,5.092396,1.780483
431,Pulisic,Chelsea,71,14.888889,67,69.610292,4.5,4.675318,2.641508,1.754616
219,David Silva,Manchester City,74,14.905660,79,69.661531,5.3,4.673495,4.431935,1.718011
344,Alli,Tottenham,89,13.035714,73,59.901737,5.6,4.595202,4.278559,1.540315
171,Maddison,Leicester City,77,19.166667,92,87.206319,4.8,4.549895,2.514490,1.639668


FORWARDS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
166,Vardy,Leicester City,101,18.947368,144,117.190871,7.6,6.185074,5.696682,1.946185
233,Rashford,Manchester Utd,91,20.000000,120,117.190871,6.0,5.859544,4.800567,1.942420
167,Iheanacho,Leicester City,57,4.000000,28,21.476349,7.0,5.369087,4.084430,2.248863
11,Aubameyang,Arsenal,108,20.166667,121,103.814523,6.0,5.147828,4.662740,1.566434
460,Abraham,Chelsea,78,18.965517,110,97.514523,5.8,5.141675,2.293197,1.841015
409,JimÃÂ©nez,Wolves,75,20.000000,104,102.329046,5.2,5.116452,5.103904,1.868264
210,AgÃÂ¼ero,Manchester City,117,14.909091,82,73.914523,5.5,4.957681,1.720027,1.449391
313,Ings,Southampton,66,19.818182,109,95.538174,5.5,4.820734,5.692971,1.876468
278,Pukki,Norwich City,65,20.000000,98,95.414523,4.9,4.770726,4.150340,1.871233


Below we compile a list sorting players based on a given metric (irrespective of position).

In [9]:
#metric = 'adjusted points per game'
metric = 'form 5'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
388,Fabianski,West Ham,49,8.000000,40,33.044848,5.0,4.130606,8.445227,1.866018
191,Salah,Liverpool,123,15.942029,110,105.599215,6.9,6.623951,8.412237,1.888707
182,Alexander-Arnold,Liverpool,74,18.928571,106,109.723582,5.6,5.796718,7.222161,2.130916
192,ManÃÂ©,Liverpool,123,18.082192,132,117.803911,7.3,6.514913,6.936672,1.857617
103,Alonso,Chelsea,60,7.966102,47,34.562261,5.9,4.338667,6.851449,1.771253
215,De Bruyne,Manchester City,105,19.041096,139,122.404331,7.3,6.428429,6.226855,1.983856
166,Vardy,Leicester City,101,18.947368,144,117.190871,7.6,6.185074,5.696682,1.946185
313,Ings,Southampton,66,19.818182,109,95.538174,5.5,4.820734,5.692971,1.876468
315,Long,Southampton,46,10.000000,21,28.076349,2.1,2.807635,5.405440,1.309067
